# Import packages

In [1]:
import pandas as pd
import numpy as np
import parse
import re
import os
import seaborn as sns
from bs4 import BeautifulSoup # handle html strings
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder

from ProcessHTML import ProcessHTML
from ExtractRooms import ExtractRooms
from GeneralizeDataset import GeneralizeDataset
from CreateInputDataset import CreateInputDataset

# Read files

In [2]:
folder = "../datasets"
paths = [os.path.join(folder, path) for path in os.listdir(folder) if "H1" in path or "H2" in path]

In [3]:
files = [pd.read_csv(path, encoding="ISO8859-1") for path in paths]

In [4]:
files[0]

,Full Address,Created,Advertised,Agreed,Completed,Date Listing Last Cancelled,Sale or Let,RTD3308_outside_space1 - Outside Space Description,EweMove Description S1 Features,EweMove Description S2 Description,...,Price / Rent,Price Qualifier,Sale Price % Achieved,Current EPC - EPC Expiry Date,DESC Council Tax Band,DESC Leasehold Ground Rent,DESC Leasehold Service Charge,ZPG_lease_expiry_years_remaining,# of Enquiry or viewings,# of Apps/Offers
0,"54 Rathmore Crescent, Churchtown, Southport PR...",03-01-2018,04-01-2018,13-02-2018,08-05-2018,NaN,Sale,Back Garden,<ul><li>A Charming Churchtown Property </li><l...,Could this Charming Churchtown property be you...,...,"<font color='blue'>&pound;200,000</font><br>",NaN,100%,14-11-2022,Band C,£4,NaN,916.0,0,1
1,"1 Glaisdale Drive, Southport PR8 6XQ",03-01-2018,06-03-2018,28-03-2018,27-04-2018,NaN,Rental,Back Garden,<ul><li>Well Proportioned Living Accommodation...,In brief the property comprises of a living ro...,...,<font color='blue'>&pound;600</font><br>Monthly,Monthly,NaN,19-11-2025,Band B,NaN,NaN,NaN,0,6
2,"3 Monica Gardens, Newbury RG14 2ET",09-01-2018,26-01-2018,05-04-2018,30-11-2018,NaN,Sale,Back Garden,<ul><li>Wonderful Family Home</li><li>Close to...,You get an awful lot of house for your money w...,...,"<font color='blue'>&pound;550,000</font><br>Gu...",Guide Price,NaN,17-09-2024,NaN,NaN,NaN,NaN,0,1
3,"190 Craven Road, NEWBURY RG14 5NS",15-01-2018,17-01-2018,27-04-2018,17-09-2018,22-01-2018,Sale,Back Garden,<ul><li>Close to Local Schools</li><li>Close t...,190 Craven Road has been fully updated over th...,...,"<font color='blue'>&pound;380,000</font><br>Of...",Offers in Region Of,NaN,08-07-2022,Band D,NaN,NaN,NaN,0,4
4,"190 Craven Road, NEWBURY RG14 5NS",22-01-2018,NaN,NaN,NaN,19-02-2018,Rental,Back Garden,<ul><li>Close to Local Schools</li><li>Close t...,190 Craven Road has been fully updated over th...,...,<font color='blue'>&pound;1</font><br>Monthly,Monthly,NaN,08-07-2022,Band D,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,"15 Ameland Road, Canvey Island SS8 9PS",07-05-2018,NaN,NaN,NaN,18-06-2018,Sale,NaN,<ul></ul>,NO EPC 15/5\r\nCouncil Tax - &pound;2117\r\nNe...,...,<font color='blue'>&pound;1</font><br>,NaN,NaN,NaN,Band E,NaN,NaN,NaN,0,0
3425,"76 The Rundels, Thundersley, Benfleet SS7 3QL",08-05-2018,29-05-2018,06-06-2018,28-06-2018,NaN,Rental,Back Garden,<ul><li>FREE TO APPLY - No 'Rip-Off' Upfront F...,"Neutrally deorated throughout, off road parkin...",...,"<font color='blue'>&pound;1,050</font><br>Monthly",Monthly,NaN,02-04-2023,Band C,NaN,NaN,NaN,0,5
3426,"76 The Rundels, Thundersley, Benfleet SS7 3QL",10-05-2018,NaN,NaN,NaN,10-05-2018,Rental,Back Garden,<ul><li>FREE TO APPLY - No 'Rip-Off' Upfront F...,"Newly decorated throughout, off road parking &...",...,"<font color='blue'>&pound;1,050</font><br>Monthly",Monthly,NaN,02-04-2023,Band C,NaN,NaN,NaN,0,0
3427,"64 Fairfield Gardens, Eastwood, Leigh on Sea S...",19-05-2018,21-05-2018,28-06-2018,25-10-2018,NaN,Sale,Back Garden,<ul></ul>,"Wonderful family home, thoughtfully extended t...",...,"<font color='blue'>&pound;350,000</font><br>",NaN,98.571428571429%,25-05-2028,Band C,NaN,NaN,NaN,0,1


In [5]:
subset = pd.read_csv("../datasets/PropertyData_wDesc.csv", encoding="ISO8859-1")

# File contents

In [6]:
col_names = None
for i, file in enumerate(files):
    if col_names is None:
        col_names = sorted(file.columns)
    else:
        if col_names != sorted(file.columns):
            print("{}: ERROR".format(paths[i]))
            break

In [7]:
for new, original in zip(col_names, sorted(subset.columns)):
    if new != original:
        print("{:40s}\n{:40s}\n".format(original, new))

Postcode                                
Full Address                            



# Missing values

In [8]:
files[0].isna().sum()

Full Address                                             0
Created                                                  0
Advertised                                             333
Agreed                                                1253
Completed                                             1375
Date Listing Last Cancelled                           2004
Sale or Let                                              1
RTD3308_outside_space1 - Outside Space Description    1390
EweMove Description S1 Features                          0
EweMove Description S2 Description                     175
EweMove Description S3 Rooms                           850
EweMove Description S4 Summary                           0
RTD3307_parking1 - Parking Description                1258
RTD3307_parking2 - Parking Description                2039
RTD3307_parking3 - Parking Description                2686
RTD3308_outside_space2 - Outside Space Description    1991
RTD3308_outside_space3 - Outside Space Description    25

# Extract postcode from full address

In [9]:
pattern = "[A-Za-z]{1,2}[0-9Rr][0-9A-Za-z]? [0-9][ABD-HJLNP-UW-Zabd-hjlnp-uw-z]{2}"

In [10]:
msg = "Currently I live in SW5 9QN, two years ago I lived in NG9 2FF"
re.findall(pattern, msg)

['SW5 9QN', 'NG9 2FF']

In [11]:
for file in files:
    postcodes = []
    for i in file["Full Address"]:
        postcode = re.findall(pattern, i)
        if len(postcode) == 0:
            postcodes.append(np.nan)
        else:
            postcodes.append(postcode[0])
    file["Full Address"] = postcodes

In [12]:
files[0]

,Full Address,Created,Advertised,Agreed,Completed,Date Listing Last Cancelled,Sale or Let,RTD3308_outside_space1 - Outside Space Description,EweMove Description S1 Features,EweMove Description S2 Description,...,Price / Rent,Price Qualifier,Sale Price % Achieved,Current EPC - EPC Expiry Date,DESC Council Tax Band,DESC Leasehold Ground Rent,DESC Leasehold Service Charge,ZPG_lease_expiry_years_remaining,# of Enquiry or viewings,# of Apps/Offers
0,PR9 8PW,03-01-2018,04-01-2018,13-02-2018,08-05-2018,NaN,Sale,Back Garden,<ul><li>A Charming Churchtown Property </li><l...,Could this Charming Churchtown property be you...,...,"<font color='blue'>&pound;200,000</font><br>",NaN,100%,14-11-2022,Band C,£4,NaN,916.0,0,1
1,PR8 6XQ,03-01-2018,06-03-2018,28-03-2018,27-04-2018,NaN,Rental,Back Garden,<ul><li>Well Proportioned Living Accommodation...,In brief the property comprises of a living ro...,...,<font color='blue'>&pound;600</font><br>Monthly,Monthly,NaN,19-11-2025,Band B,NaN,NaN,NaN,0,6
2,RG14 2ET,09-01-2018,26-01-2018,05-04-2018,30-11-2018,NaN,Sale,Back Garden,<ul><li>Wonderful Family Home</li><li>Close to...,You get an awful lot of house for your money w...,...,"<font color='blue'>&pound;550,000</font><br>Gu...",Guide Price,NaN,17-09-2024,NaN,NaN,NaN,NaN,0,1
3,RG14 5NS,15-01-2018,17-01-2018,27-04-2018,17-09-2018,22-01-2018,Sale,Back Garden,<ul><li>Close to Local Schools</li><li>Close t...,190 Craven Road has been fully updated over th...,...,"<font color='blue'>&pound;380,000</font><br>Of...",Offers in Region Of,NaN,08-07-2022,Band D,NaN,NaN,NaN,0,4
4,RG14 5NS,22-01-2018,NaN,NaN,NaN,19-02-2018,Rental,Back Garden,<ul><li>Close to Local Schools</li><li>Close t...,190 Craven Road has been fully updated over th...,...,<font color='blue'>&pound;1</font><br>Monthly,Monthly,NaN,08-07-2022,Band D,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,SS8 9PS,07-05-2018,NaN,NaN,NaN,18-06-2018,Sale,NaN,<ul></ul>,NO EPC 15/5\r\nCouncil Tax - &pound;2117\r\nNe...,...,<font color='blue'>&pound;1</font><br>,NaN,NaN,NaN,Band E,NaN,NaN,NaN,0,0
3425,SS7 3QL,08-05-2018,29-05-2018,06-06-2018,28-06-2018,NaN,Rental,Back Garden,<ul><li>FREE TO APPLY - No 'Rip-Off' Upfront F...,"Neutrally deorated throughout, off road parkin...",...,"<font color='blue'>&pound;1,050</font><br>Monthly",Monthly,NaN,02-04-2023,Band C,NaN,NaN,NaN,0,5
3426,SS7 3QL,10-05-2018,NaN,NaN,NaN,10-05-2018,Rental,Back Garden,<ul><li>FREE TO APPLY - No 'Rip-Off' Upfront F...,"Newly decorated throughout, off road parking &...",...,"<font color='blue'>&pound;1,050</font><br>Monthly",Monthly,NaN,02-04-2023,Band C,NaN,NaN,NaN,0,0
3427,SS9 5SF,19-05-2018,21-05-2018,28-06-2018,25-10-2018,NaN,Sale,Back Garden,<ul></ul>,"Wonderful family home, thoughtfully extended t...",...,"<font color='blue'>&pound;350,000</font><br>",NaN,98.571428571429%,25-05-2028,Band C,NaN,NaN,NaN,0,1


In [13]:
file = files[0]
file = file.rename(columns={"Full Address": "Postcode"})
file = file[file["Postcode"].notna()]
file = file.rename(index={i: j for i, j in zip(file.index, range(len(file)))})
file

,Postcode,Created,Advertised,Agreed,Completed,Date Listing Last Cancelled,Sale or Let,RTD3308_outside_space1 - Outside Space Description,EweMove Description S1 Features,EweMove Description S2 Description,...,Price / Rent,Price Qualifier,Sale Price % Achieved,Current EPC - EPC Expiry Date,DESC Council Tax Band,DESC Leasehold Ground Rent,DESC Leasehold Service Charge,ZPG_lease_expiry_years_remaining,# of Enquiry or viewings,# of Apps/Offers
0,PR9 8PW,03-01-2018,04-01-2018,13-02-2018,08-05-2018,NaN,Sale,Back Garden,<ul><li>A Charming Churchtown Property </li><l...,Could this Charming Churchtown property be you...,...,"<font color='blue'>&pound;200,000</font><br>",NaN,100%,14-11-2022,Band C,£4,NaN,916.0,0,1
1,PR8 6XQ,03-01-2018,06-03-2018,28-03-2018,27-04-2018,NaN,Rental,Back Garden,<ul><li>Well Proportioned Living Accommodation...,In brief the property comprises of a living ro...,...,<font color='blue'>&pound;600</font><br>Monthly,Monthly,NaN,19-11-2025,Band B,NaN,NaN,NaN,0,6
2,RG14 2ET,09-01-2018,26-01-2018,05-04-2018,30-11-2018,NaN,Sale,Back Garden,<ul><li>Wonderful Family Home</li><li>Close to...,You get an awful lot of house for your money w...,...,"<font color='blue'>&pound;550,000</font><br>Gu...",Guide Price,NaN,17-09-2024,NaN,NaN,NaN,NaN,0,1
3,RG14 5NS,15-01-2018,17-01-2018,27-04-2018,17-09-2018,22-01-2018,Sale,Back Garden,<ul><li>Close to Local Schools</li><li>Close t...,190 Craven Road has been fully updated over th...,...,"<font color='blue'>&pound;380,000</font><br>Of...",Offers in Region Of,NaN,08-07-2022,Band D,NaN,NaN,NaN,0,4
4,RG14 5NS,22-01-2018,NaN,NaN,NaN,19-02-2018,Rental,Back Garden,<ul><li>Close to Local Schools</li><li>Close t...,190 Craven Road has been fully updated over th...,...,<font color='blue'>&pound;1</font><br>Monthly,Monthly,NaN,08-07-2022,Band D,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,SS8 9PS,07-05-2018,NaN,NaN,NaN,18-06-2018,Sale,NaN,<ul></ul>,NO EPC 15/5\r\nCouncil Tax - &pound;2117\r\nNe...,...,<font color='blue'>&pound;1</font><br>,NaN,NaN,NaN,Band E,NaN,NaN,NaN,0,0
3412,SS7 3QL,08-05-2018,29-05-2018,06-06-2018,28-06-2018,NaN,Rental,Back Garden,<ul><li>FREE TO APPLY - No 'Rip-Off' Upfront F...,"Neutrally deorated throughout, off road parkin...",...,"<font color='blue'>&pound;1,050</font><br>Monthly",Monthly,NaN,02-04-2023,Band C,NaN,NaN,NaN,0,5
3413,SS7 3QL,10-05-2018,NaN,NaN,NaN,10-05-2018,Rental,Back Garden,<ul><li>FREE TO APPLY - No 'Rip-Off' Upfront F...,"Newly decorated throughout, off road parking &...",...,"<font color='blue'>&pound;1,050</font><br>Monthly",Monthly,NaN,02-04-2023,Band C,NaN,NaN,NaN,0,0
3414,SS9 5SF,19-05-2018,21-05-2018,28-06-2018,25-10-2018,NaN,Sale,Back Garden,<ul></ul>,"Wonderful family home, thoughtfully extended t...",...,"<font color='blue'>&pound;350,000</font><br>",NaN,98.571428571429%,25-05-2028,Band C,NaN,NaN,NaN,0,1


# Test compatibility with CreateInputDataset

In [14]:
files = [pd.read_csv(path, encoding="ISO8859-1") for path in paths]

In [15]:
creation = CreateInputDataset(files[8])

In [16]:
creation.get_general_dataset()

,Postcode,Sale or Let,Price Qualifier,DESC Council Tax Band,RTD3316_condition1 - Condition Description,# of Enquiry or viewings,# of Apps/Offers
9,276,1,4,6,0,0,1
33,216,1,4,5,0,0,0
49,479,0,3,0,0,0,0
57,482,0,3,1,0,0,0
83,483,0,3,2,0,0,3
...,...,...,...,...,...,...,...
3075,579,0,3,2,0,0,0
3076,579,1,6,2,0,0,2
3078,68,0,3,2,0,0,4
3079,587,0,3,2,0,0,2


In [17]:
creation.get_room_dataset()

,bedroom number,kitchen number,living number,bathroom number,dining number,other number
9,5,1,1,4,2,10
33,2,1,0,2,1,15
49,2,1,1,1,1,2
57,4,1,1,1,1,4
83,3,1,1,1,1,4
...,...,...,...,...,...,...
3075,3,1,0,1,1,8
3076,3,1,0,1,1,5
3078,3,1,0,1,1,4
3079,3,1,0,2,1,7


In [18]:
result = creation.get_categorical_dataset()
result

,Allocated,Communal,Covered,Driveway,Garage,Gated,Off Street,On Street,Permit,Private,...,Central,Double Glazing,Eco-Friendly,Electric,Gas,Gas Central,Night Storage,Oil,Solar,Under Floor
9,0,0,0,1,0,1,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0
33,0,0,0,1,1,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
49,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0
57,0,0,0,1,1,0,1,0,0,0,...,1,1,0,0,0,1,0,0,0,0
83,0,0,0,1,0,0,1,0,0,0,...,1,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3076,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
3078,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3079,0,0,0,1,1,0,1,0,0,0,...,0,1,0,0,1,1,0,0,0,0


In [19]:
creation.get_labels()

,Completed,Price / Rent
9,1,900000.0
33,0,350000.0
49,0,600.0
57,0,875.0
83,1,675.0
...,...,...
3075,0,1.0
3076,1,275000.0
3078,1,1350.0
3079,1,1325.0


# Create input dataset

In [20]:
folder = "../datasets"
paths = [os.path.join(folder, path) for path in os.listdir(folder) if "final" not in path]

In [23]:
features, labels, sources = CreateInputDataset.create_dataset(*paths)

In [24]:
features

,Postcode,Sale or Let,Price Qualifier,DESC Council Tax Band,RTD3316_condition1 - Condition Description,# of Enquiry or viewings,# of Apps/Offers,bedroom number,kitchen number,living number,...,Double Glazing,Eco-Friendly,Electric,Gas,Gas Central,Night Storage,Oil,Solar,Solar Water,Under Floor
1,3633,0,3,1,0,0,6,2,1,1,...,1,0,0,0,1,0,0,0,0,0
3,3672,1,7,3,0,0,4,3,1,0,...,0,0,0,0,1,0,0,0,0,0
4,3672,0,3,3,0,0,0,3,1,0,...,0,0,0,0,1,0,0,0,0,0
5,3672,0,3,3,0,0,0,3,1,0,...,0,0,0,0,1,0,0,0,0,0
8,3675,1,7,4,0,0,1,4,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34571,4510,0,3,3,0,0,0,3,1,2,...,0,0,0,0,1,0,0,0,0,0
34574,4476,1,6,3,0,0,1,3,1,1,...,1,0,0,0,1,0,0,0,0,0
34576,4491,1,6,5,0,0,0,4,1,0,...,1,0,0,1,1,0,0,0,0,0
34578,550,0,3,0,0,0,2,1,1,0,...,1,0,0,1,1,0,0,0,0,0


In [25]:
sources

1                 ../datasets/H1 2018 Data.csv@1
3                 ../datasets/H1 2018 Data.csv@3
4                 ../datasets/H1 2018 Data.csv@4
5                 ../datasets/H1 2018 Data.csv@5
8                 ../datasets/H1 2018 Data.csv@8
                          ...                   
34571    ../datasets/PropertyData_wDesc.csv@3639
34574    ../datasets/PropertyData_wDesc.csv@3642
34576    ../datasets/PropertyData_wDesc.csv@3644
34578    ../datasets/PropertyData_wDesc.csv@3646
34579    ../datasets/PropertyData_wDesc.csv@3647
Name: Source, Length: 9177, dtype: object

In [26]:
labels

,Completed,Price / Rent
1,1,600.0
3,1,380000.0
4,0,1.0
5,0,1.0
8,1,475000.0
...,...,...
34571,0,1200.0
34574,1,340000.0
34576,0,450000.0
34578,1,750.0


In [27]:
features.to_csv("../datasets/final_features.csv", index=False)
labels.to_csv("../datasets/final_labels.csv", index=False)
sources.to_csv("../datasets/final_sources.csv", index=False)